# TIME SERIES FORECASTING AND SENTIMENT ANALYSIS OF BIG DATA PROCESSED WITH SparkSQL VS HBASE, CASSANDRA, MONGODB

# Dataset
The dataste is a large dataset gleaned from the twitter API that is called ProjectTweets.csv.

This dataset contains 1,600,000 tweets extracted using the twitter api. 


Content
It contains the following 5 fields:
- ids: The id of the tweet (eg. 4587)
- date: the date of the tweet (eg. Sat May 16 23:58:44 UTC 2009)
- flag: The query (eg. lyx). If there is no query, then this value is NO_QUERY.
- user: the user that tweeted (eg. bobthebuilder)
- text: the text of the tweet (eg. Lyx is cool)

# STOP ALL ACTIVE SPARK SESSIONS

In [1]:
from pyspark.sql import SparkSession
#Initialize SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()
spark.stop()

24/04/29 18:05:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import StructType, StringType, TimestampType
from pyspark.sql.functions import col

# Loading Data from local machine to MongoDB

One can read  CSV file into DataFrame from hadoop

csv_path = "hdfs://localhost:9000/ProjectTweets/ProjectTweets.csv"

df = spark.read.csv(csv_path, header=False, inferSchema=True)

# Step One: Populating the ProjectTweets CSV Into SparkSQL

This involves adding or inserting the ProjectTweets CSV into SparkSQL.

## Step 1: Initialize Spark Session and Read the ProjectTweets CSV

In [3]:
# Initialize the Spark Session
spark = SparkSession.builder.appName("ProjectTweets CSV to SparkSQL").getOrCreate()

# Set legacy timeParserPolicy
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Define the schema for the csv file 
schema = StructType().add("_c0", StringType(), True).add("_c1", StringType(), True).add("_c2", StringType(), True).add("_c3", StringType(), True).add("_c4", StringType(), True).add("_c5", StringType(), True)

# Read the CSV into a DataFrame called df
df = spark.read.format("csv").option("header", "false").schema(schema).load("file:///home/hduser/ProjectTweets.csv")

# Rename the headers
df = df.withColumnRenamed("_c0", "PRIMARY KEY").withColumnRenamed("_c1", "ID").withColumnRenamed("_c2", "date").withColumnRenamed("_c3", "flag").withColumnRenamed("_c4", "user").withColumnRenamed("_c5", "text")

# Convert string date to TimestampType
df = df.withColumn("date", to_timestamp(df["date"], "EEE MMM dd HH:mm:ss zzzz yyyy"))

# Print schema
df.printSchema()

# Show DataFrame
df.show(5)



root
 |-- PRIMARY KEY: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- flag: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)



+-----------+----------+-------------------+--------+---------------+--------------------+
|PRIMARY KEY|        ID|               date|    flag|           user|                text|
+-----------+----------+-------------------+--------+---------------+--------------------+
|          0|1467810369|2009-04-07 06:19:45|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|          1|1467810672|2009-04-07 06:19:49|NO_QUERY|  scotthamilton|is upset that he ...|
|          2|1467810917|2009-04-07 06:19:53|NO_QUERY|       mattycus|@Kenichan I dived...|
|          3|1467811184|2009-04-07 06:19:57|NO_QUERY|        ElleCTF|my whole body fee...|
|          4|1467811193|2009-04-07 06:19:57|NO_QUERY|         Karoli|@nationwideclass ...|
+-----------+----------+-------------------+--------+---------------+--------------------+
only showing top 5 rows



In [4]:
#View if the columns have been renamed
df.printSchema()

root
 |-- PRIMARY KEY: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- flag: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)



In [5]:
#View the Spark DataFrame
df.show(5)

+-----------+----------+-------------------+--------+---------------+--------------------+
|PRIMARY KEY|        ID|               date|    flag|           user|                text|
+-----------+----------+-------------------+--------+---------------+--------------------+
|          0|1467810369|2009-04-07 06:19:45|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|          1|1467810672|2009-04-07 06:19:49|NO_QUERY|  scotthamilton|is upset that he ...|
|          2|1467810917|2009-04-07 06:19:53|NO_QUERY|       mattycus|@Kenichan I dived...|
|          3|1467811184|2009-04-07 06:19:57|NO_QUERY|        ElleCTF|my whole body fee...|
|          4|1467811193|2009-04-07 06:19:57|NO_QUERY|         Karoli|@nationwideclass ...|
+-----------+----------+-------------------+--------+---------------+--------------------+
only showing top 5 rows



# Step Two: Exploratory Data Analysis (EDA)

## Checking for Duplicates (based on ID, user and text) and Missing data

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum  # Import the 'col' and 'sum' functions

# Initialize Spark Session
spark = SparkSession.builder.appName("Duplicate and Missing Data Check").getOrCreate()

# Define the columns to check for duplicates and missing values
columns_to_check = ['PRIMARY KEY','ID', 'user', 'text']

# Create a pipeline to check for duplicates and missing values
pipeline_df = df

# Step 1: Remove duplicate records based on specified columns
pipeline_df = pipeline_df.dropDuplicates(subset=columns_to_check)

# Step 2: Check for missing values
missing_counts = pipeline_df.select([col(c).isNull().cast("int").alias(c) for c in columns_to_check]).agg(*[sum(c).alias(c) for c in columns_to_check]).collect()[0]

# Print the results
print("Number of duplicate records removed:", df.count() - pipeline_df.count())

print("Missing value counts:")
for col_name, missing_count in zip(columns_to_check, missing_counts):
    print(col_name, missing_count)



In [ ]:
#Checking for duplicates in ID and user name
# Count the number of rows before removing duplicates
count_before = df.count()

# Remove duplicates
df_no_duplicates = df.dropDuplicates()

# Count the number of rows after removing duplicates
count_after = df_no_duplicates.count()

# Calculate the number of duplicates
num_duplicates = count_before - count_after

print(f"Number of duplicate rows removed: {num_duplicates}")


In [ ]:
# Count the number of rows before removing duplicates
count_before = df.count()

# Remove duplicates based on a specific column
df_no_duplicates = df.dropDuplicates(subset=['user'])

# Count the number of rows after removing duplicates
count_after = df_no_duplicates.count()

# Calculate the number of duplicates
num_duplicates = count_before - count_after

print(f"Number of duplicate rows removed based on user: {num_duplicates}")


## Summary Statistics

In [ ]:
#Summary Statistics
df.describe().show()

## EXTRACTING TIME COMPONENTS

In [ ]:
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second


#Initialize Spark Session
spark = SparkSession.builder.appName("DateTime Visualization").getOrCreate()

# Extract relevant time components including hours, minutes, and seconds
df = df.withColumn("year", year("date")).withColumn("month", month("date")).withColumn("day", dayofmonth("date")).withColumn("hour", hour("date")).withColumn("minute", minute("date")).withColumn("second", second("date"))

# Aggregate data
time_series_data = df.groupBy("date","year", "month", "day", "hour", "minute", "second").count().orderBy("year", "month", "day", "hour", "minute", "second")

In [ ]:
#View the df DataFrame after extracting time components
df.show(5)

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.functions import count

# Group by the year variable and count the occurrences
year_counts = df.groupBy("year").agg(count("*").alias("count")).orderBy("year")

# Show the tabulated counts
year_counts.show()


In [ ]:
from pyspark.sql.functions import count

# Group by the month variable and count the occurrences
month_counts = df.groupBy("month").agg(count("*").alias("count")).orderBy("month")

# Show the tabulated counts
month_counts.show()

In [ ]:
from pyspark.sql.functions import count

# Group by the day variable and count the occurrences
day_counts = df.groupBy("day").agg(count("*").alias("count")).orderBy("day")

# Show the tabulated counts
day_counts.show()

In [ ]:
from pyspark.sql.functions import count

# Group by the day variable and count the occurrences
ID_counts = df.groupBy("ID").agg(count("*").alias("count")).orderBy("ID")

# Show the tabulated counts
ID_counts.show()

In [ ]:
time_series_data.show(5)

In [ ]:
type(time_series_data)

In [ ]:
#!pip install nltk
import nltk
nltk.download('vader_lexicon')

In [ ]:
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize Spark Session
spark = SparkSession.builder.appName("Text Analysis").getOrCreate()

# Tokenize text using a regular expression tokenizer
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern=r'\W')
df = tokenizer.transform(df)

# Initialize CountVectorizer
cv = CountVectorizer(inputCol="words", outputCol="features", minDF=1, vocabSize=1000)
model = cv.fit(df)
df = model.transform(df)

# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Define a UDF to analyze sentiment using VADER
def analyze_sentiment(text):
    sentiment = sid.polarity_scores(text)
    if sentiment['compound'] >= 0.05:
        return 'Positive'
    elif sentiment['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

sentiment_udf = udf(analyze_sentiment, StringType())

# Apply the UDF to analyze sentiment for each comment
df = df.withColumn("sentiment", sentiment_udf(df["text"]))

# Show the DataFrame
df.select("text", "date", "sentiment", "features").show(5)


In [ ]:
df.show(1)

In [ ]:
#!pip install statsmodels
!pip install plotly

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, date_format
from pyspark.sql.types import StringType
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objects as go
import pandas as pd

# Initialize Spark Session
spark = SparkSession.builder.appName("Sentiment Forecast Dashboard").getOrCreate()

# Step 1: Preprocess the data
sid = SentimentIntensityAnalyzer()
analyze_sentiment_udf = udf(lambda text: sid.polarity_scores(text)['compound'], StringType())
df = df.withColumn("sentiment_score", analyze_sentiment_udf(df["text"]))
df = df.withColumn("date_str", date_format("date", "yyyy-MM-dd"))
df_agg = df.groupBy("date_str").agg({"sentiment_score": "mean"}).withColumnRenamed("avg(sentiment_score)", "sentiment_score")

# Step 2: Train a time series forecasting model
# Convert Spark DataFrame to Pandas DataFrame
df_pd = df_agg.toPandas()
df_pd['date'] = pd.to_datetime(df_pd['date_str'])

# Step 3: Make forecasts
# Assuming ARIMA model for simplicity
model = ARIMA(df_pd['sentiment_score'], order=(1,1,1))
fit_model = model.fit()
forecast_1_day = fit_model.forecast(steps=1)
forecast_3_days = fit_model.forecast(steps=3)
forecast_7_days = fit_model.forecast(steps=7)

# Step 4: Display the forecasts (using Plotly for visualization)

# Plot historical sentiment data
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_pd['date'], y=df_pd['sentiment_score'], mode='lines', name='Historical Sentiment'))

# Plot forecasted values
forecast_dates = [df_pd['date'].iloc[-1] + pd.Timedelta(days=i) for i in range(1, 8)]
fig.add_trace(go.Scatter(x=forecast_dates, y=forecast_1_day, mode='lines', name='Forecast (1 day)'))
fig.add_trace(go.Scatter(x=forecast_dates[:3], y=forecast_3_days, mode='lines', name='Forecast (3 days)'))
fig.add_trace(go.Scatter(x=forecast_dates[:7], y=forecast_7_days, mode='lines', name='Forecast (7 days)'))

# Update layout
fig.update_layout(title='Sentiment Forecast Dashboard',
                  xaxis_title='Date',
                  yaxis_title='Sentiment Score')

# Show the dashboard
fig.show()


In [ ]:
# Plot historical sentiment data
fig_historical = go.Figure()
fig_historical.add_trace(go.Scatter(x=df_pd['date'], y=df_pd['sentiment_score'], mode='lines', name='Historical Sentiment'))
fig_historical.update_layout(title='Historical Sentiment',
                             xaxis_title='Date',
                             yaxis_title='Sentiment Score')

# Plot forecasted values for 1 day
fig_1_day = go.Figure()
fig_1_day.add_trace(go.Scatter(x=forecast_dates, y=forecast_1_day, mode='lines', name='Forecast (1 day)'))
fig_1_day.update_layout(title='Sentiment Forecast (1 day)',
                        xaxis_title='Date',
                        yaxis_title='Sentiment Score')

# Plot forecasted values for 3 days
fig_3_days = go.Figure()
fig_3_days.add_trace(go.Scatter(x=forecast_dates[:3], y=forecast_3_days, mode='lines', name='Forecast (3 days)'))
fig_3_days.update_layout(title='Sentiment Forecast (3 days)',
                         xaxis_title='Date',
                         yaxis_title='Sentiment Score')

# Plot forecasted values for 7 days
fig_7_days = go.Figure()
fig_7_days.add_trace(go.Scatter(x=forecast_dates[:7], y=forecast_7_days, mode='lines', name='Forecast (7 days)'))
fig_7_days.update_layout(title='Sentiment Forecast (7 days)',
                         xaxis_title='Date',
                         yaxis_title='Sentiment Score')

# Show the separate figures
fig_historical.show()
fig_1_day.show()
fig_3_days.show()
fig_7_days.show()


In [ ]:
import plotly.graph_objects as go
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, date_format
from pyspark.sql.types import StringType
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

# Initialize Spark Session
spark = SparkSession.builder.appName("Sentiment Forecast Dashboard").getOrCreate()

# Assuming df is your Spark DataFrame containing the sentiment data

# Step 1: Preprocess the data
sid = SentimentIntensityAnalyzer()
analyze_sentiment_udf = udf(lambda text: sid.polarity_scores(text)['compound'], StringType())
df = df.withColumn("sentiment_score", analyze_sentiment_udf(df["text"]))
df = df.withColumn("month", date_format("date", "MM"))
df = df.withColumn("day", date_format("date", "dd"))
df_agg = df.groupBy("month", "day").agg({"sentiment_score": "mean"}).withColumnRenamed("avg(sentiment_score)", "sentiment_score")

# Step 2: Train a time series forecasting model
# Convert Spark DataFrame to Pandas DataFrame
df_pd = df_agg.toPandas()
df_pd['date'] = pd.to_datetime(df_pd['month'] + '-' + df_pd['day'])

# Step 3: Make forecasts
# Assuming ARIMA model for simplicity
model = ARIMA(df_pd['sentiment_score'], order=(1,1,1))
fit_model = model.fit()
forecast_1_day = fit_model.forecast(steps=1)
forecast_3_days = fit_model.forecast(steps=3)
forecast_7_days = fit_model.forecast(steps=7)

# Step 4: Display the forecasts (using Plotly for visualization)

# Create a dropdown menu for selecting months
month_buttons = [
    {'label': f'Month {month}', 'method': 'update', 'args': [{'visible': [df_pd['month'] == month]}]}
    for month in df_pd['month'].unique()
]

# Plot historical sentiment data
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_pd['day'], y=df_pd['sentiment_score'], mode='lines', name='Historical Sentiment'))

# Plot forecasted values
forecast_dates = [df_pd['date'].iloc[-1] + pd.Timedelta(days=i) for i in range(1, 8)]
fig.add_trace(go.Scatter(x=forecast_dates, y=forecast_1_day, mode='lines', name='Forecast (1 day)'))
fig.add_trace(go.Scatter(x=forecast_dates[:3], y=forecast_3_days, mode='lines', name='Forecast (3 days)'))
fig.add_trace(go.Scatter(x=forecast_dates[:7], y=forecast_7_days, mode='lines', name='Forecast (7 days)'))

# Update layout
fig.update_layout(title='Sentiment Forecast Dashboard',
                  xaxis_title='Day',
                  yaxis_title='Sentiment Score',
                  updatemenus=[{'buttons': month_buttons, 'direction': 'down', 'showactive': True, 'x': 0.5, 'xanchor': 'center'}],
                  showlegend=True)

# Show the dashboard
fig.show()

# Populated the Project Tweets CSV  into the NoSQL DATABASE (MONGODB) USING SPARK

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Process data from MongoDB using spark").config("spark.mongodb.input.uri", "mongodb://localhost:27017/project.ProjectTweets").config("spark.mongodb.output.uri", "mongodb://localhost:27017/Project.my_output_ProjectTweets").getOrCreate()

# Read data from MongoDB into a DataFrame
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [ ]:
# Perform Spark transformations and actions on the DataFrame
# For example:
df_processed = df.select("field1", "field2").filter(df["field1"] > 10)

# Write processed data back to MongoDB
df_processed.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").save()

# Stop Spark session
spark.stop()